In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import GridSearchCV


(X_train, y_train), (X_test, y_test) = mnist.Age_Dataset()


img_rows, img_cols = 1009, 9
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255


num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)


def create_model(num_filters=32, kernel_size=(3,3), activation='relu', dropout_rate=0.2):
    model = Sequential()
    model.add(Conv2D(num_filters, kernel_size=kernel_size, activation=activation, input_shape=input_shape))
    model.add(Conv2D(num_filters, kernel_size=kernel_size, activation=activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(128, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, verbose=0)


param_grid = {
    'num_filters': [16, 32, 64],
    'kernel_size': [(3,3), (5,5), (7,7)],
    'activation': ['relu', 'tanh', 'sigmoid'],
    'dropout_rate': [0.2, 0.3, 0.4]
}


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)


grid_result = grid.fit(X_train, y_train)


print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


best_model = create_model(num_filters=grid_result.best_params_['num_filters'],
                          kernel_size=grid_result.best_params_['kernel_size'],
                          activation=grid_result.best_params_['activation'],
                          dropout_rate=grid_result.best_params_['
